Xanadu Quantum Codebook - Solutions

By arvizu-god

# I.2 Quantum Circuits

Now that you've run your very first quantum algorithm, you might be wondering whether there's a cleaner way of representing things than vectors and matrix multiplication. In the next few exercises, we'll abstract away the linear algebra, and explore how to use PennyLane to implement quantum algorithms. In tandem, you'll start playing with quantum circuits.

Quantum circuits are a common means of visually representing the sequence of operations that are performed on qubits during a quantum computation. They consist of a set of operations, or gates, applied to a set of qubits (or more generally, wires). Each wire in the diagram represents a qubit. Circuits are read from left to right, and this is the order in which operations are applied. Quantum circuits end in a measurement of one or more qubits. For example, the circuit below has 3 qubits that start in state $|0\rangle$, applies 5 operations, and measures every qubit at the end.

<p align="center">
(https://codebook.xanadu.ai/images/circuit_i-2-1.svg "circuit_i-2-1")
</p>

In PennyLane, a quantum circuit is represented by a quantum function. These are just regular Python functions, with some special properties: they must apply one or more quantum operations, and return one or more quantum measurements. Expressing quantum circuits as quantum functions allows us to represent circuits compactly, and use regular programming concepts such as subroutines (subcircuits), loops, and input parameters.

Suppose we would like to write a circuit for 2 qubits. By default in PennyLane, qubits (wires) are ordered numerically starting from 0 (which corresponds to the top qubit in the circuit). In pseudocode, a quantum function looks something like this:

```
import pennylane as qml

def my_quantum_function(params):

    # Single-qubit operations with no input parameters
    qml.Gate1(wires=0)
    qml.Gate2(wires=1)

    # A single-qubit operation with an input parameter
    qml.Gate3(params[0], wires=0)

    # Two-qubit operation with no input parameter on wires 0 and 1
    qml.TwoQubitGate1(wires=[0, 1])

    # Two-qubit operation with an input parameter on wires 0 and 1
    qml.TwoQubitGate2(params[1], wires=[0, 1])

    # Return the result of a measurement
    return qml.Measurement(wires=[0, 1])
```

You can see how operations on qubits are applied one per line. Each operation indicates which qubits it acts on, and some operations also take input parameters.

Tip. PennyLane uses the term wires rather than qubits because it also supports continuous-variable quantum computing, for which a wire in a circuit does not correspond to a qubit.

## Codercise I.2.1

The code below is a quantum function with all the gates from the above circuit (which we reproduce here for convenience). However, the gates are out of order! Re-arrange the lines of the function to match the order of operations in the circuit.

In [1]:
import pennylane as qml

In [2]:
def my_circuit(theta, phi): 
    ##################
    # YOUR CODE HERE #
    ##################

    # REORDER THESE 5 GATES TO MATCH THE CIRCUIT IN THE PICTURE

    qml.CNOT(wires=[0, 1])
    qml.RX(theta, wires=2)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[2, 0])
    qml.RY(phi, wires=1)
    
    # This is the measurement; we return the probabilities of all possible output states
    # You'll learn more about what types of measurements are available in a later node
    return qml.probs(wires=[0, 1, 2])

While quantum circuits are represented as quantum functions, a quantum function alone isn't enough to run and execute a circuit. For this we need two extra parts:
* a device to run the circuit on
* a QNode, which binds the circuit to the device, and executes it

In this book, our devices will be quantum simulators, but PennyLane provides plugins that enable us to run on real quantum hardware as well! To construct a device in PennyLane, we need to know the name or type of the device, and the number of qubits (wires) it has:
```
dev = qml.device('device.name', wires=num_qubits)
```
In this section, we will always be using the $\textbf{default.qubit device}$, which is a standard quantum simulator. You can also give string labels to the wires on a device.
```
dev = qml.device('default.qubit', wires=["wire_a", "wire_b"])
```
Once we have a device, we can construct a QNode. QNodes are the main unit of quantum computation in PennyLane. There are two ways to construct a QNode from a device and a quantum function. The first, which you will see in the next exercise, is using the qml.QNode function:
```
my_qnode = qml.QNode(my_circuit, my_device)
```
Once a QNode is created, it can be called like a function using the same parameters as the quantum function upon which it's built.

## Codercise I.2.2

Complete the quantum function in the PennyLane code below to implement the following quantum circuit. We'll then construct a QNode, and run the circuit on the provided device.

In [3]:
# This creates a device with three wires on which PennyLane can run computations
dev = qml.device("default.qubit", wires=3)


def my_circuit(theta, phi, omega):

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE CIRCUIT BY ADDING THE GATES

    # Here are two examples, so you can see the format:
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega,wires=2)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,0])

    return qml.probs(wires=[0, 1, 2])


# This creates a QNode, binding the function and device
my_qnode = qml.QNode(my_circuit, dev)

# We set up some values for the input parameters
theta, phi, omega = 0.1, 0.2, 0.3

# Now we can execute the QNode by calling it like we would a regular function
my_qnode(theta, phi, omega)

tensor([9.87560268e-01, 0.00000000e+00, 0.00000000e+00, 2.47302134e-03,
        2.48960206e-05, 0.00000000e+00, 0.00000000e+00, 9.94181506e-03], requires_grad=True)

Tip. At this point, you might be wondering what these operations are being applied to; after all, quantum functions are simply a list of operations, and there is no quantum state present like there was in our single-qubit simulator from earlier. All the linear algebra, and maintenance of state, happens under the hood in PennyLane on the devices. This enables you to focus more on your quantum algorithm, and less on the implementation details!

The second way to construct a QNode in PennyLane is using a decorator. Decorating a quantum function with @qml.qnode(dev) will automatically produce a QNode with the same name as your function that can be run on the device dev. Try it below!

## Codercise I.2.3

The quantum function below implements the circuit from the previous exercise. Apply a decorator to the quantum function to construct a QNode, then run it using the provided input parameters.

In [4]:
dev = qml.device("default.qubit", wires=3)

##################
# YOUR CODE HERE #
##################
@qml.qnode(dev)
# DECORATE THE FUNCTION BELOW TO TURN IT INTO A QNODE

def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    return qml.probs(wires=[0, 1, 2])


theta, phi, omega = 0.1, 0.2, 0.3

##################
# YOUR CODE HERE #
##################

# RUN THE QNODE WITH THE PROVIDED PARAMETERS
my_circuit(theta,phi,omega)

tensor([9.87560268e-01, 0.00000000e+00, 0.00000000e+00, 2.47302134e-03,
        2.48960206e-05, 0.00000000e+00, 0.00000000e+00, 9.94181506e-03], requires_grad=True)

Quantum circuits are algorithms. Just like how we profile code, and study the complexity and resource requirements of regular algorithms and software, we can do the same for quantum circuits to make sure we implement them as efficiently as possible. The number of gates, and the type of gates, are useful metrics. However, one particularly important metric is that of circuit depth. Loosely, the depth is the number of time steps it takes for a circuit to run, if we do things as in-parallel as possible. Alternatively, you can think of it as the number of layers in a circuit.

## Codercise I.2.4

What is the depth of the circuit in the picture above?

In [5]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    return qml.probs(wires=[0, 1, 2])


##################
# YOUR CODE HERE #
##################

# FILL IN THE CORRECT CIRCUIT DEPTH
depth = 4

That's the end of this Xanadu's Quantum Codebook codercise I.2! See you in the next one. Goodbye!